In [3]:
import pandas as pd
from tqdm import tqdm

file_path = "ol_dump_ratings_2025-01-08.txt"  # Change to your actual file path
output_file = "openlibrary_ratings.csv"

data_list = []

with open(file_path, "r", encoding="utf-8") as file:
    for line in tqdm(file, desc="Processing Ratings"):
        parts = line.strip().split("\t")

        if len(parts) < 3:
            continue  # Skip malformed lines

        work_id = parts[0].split("/")[-1]  # Extract Work ID (OLxxxxW)
        book_id = None  # Default to None

        if parts[1].startswith("/books/"):
            book_id = parts[1].split("/")[-1]  # Extract Book ID (OLxxxxM)
            rating_str = parts[2]
            timestamp = parts[3]
        else:
            rating_str = parts[1]
            timestamp = parts[2]

        # Handle missing or empty ratings safely
        try:
            rating = int(rating_str) if rating_str.strip() else None
        except ValueError:
            rating = None  # Assign None if conversion fails

        data_list.append((work_id, book_id, rating, timestamp))

df = pd.DataFrame(data_list, columns=["Work_ID", "Book_ID", "Rating", "Timestamp"])

df.to_csv(output_file, index=False)

Processing Ratings: 509907it [00:00, 523999.49it/s]


In [4]:
df

,Work_ID,Book_ID,Rating,Timestamp
0,OL17882343W,None,NaN,3
1,OL1629179W,OL22981670M,5.0,2018-06-20
2,OL4226036W,OL10690412M,5.0,2018-06-20
3,OL5264255W,OL2719185M,5.0,2018-06-20
4,OL1681415W,OL2582724M,5.0,2018-06-20
...,...,...,...,...
509902,OL461939W,OL8494161M,3.0,2022-07-30
509903,OL1708091W,OL10691251M,3.0,2022-07-30
509904,OL8299102W,OL7695147M,4.0,2022-07-30
509905,OL15191772W,OL24324060M,4.0,2022-07-30


In [5]:
# summarize the rating data for each work
work_ratings = df.groupby("Work_ID")["Rating"].agg(["count", "mean", "std"]).reset_index()

In [6]:
work_ratings

,Work_ID,count,mean,std
0,OL10000000W,1,3.000000,NaN
1,OL100001W,1,1.000000,NaN
2,OL1000035W,1,4.000000,NaN
3,OL1000043W,1,3.000000,NaN
4,OL100004W,1,3.000000,NaN
...,...,...,...,...
242053,OL99997W,1,4.000000,NaN
242054,OL999981W,3,3.666667,1.154701
242055,OL999982W,1,5.000000,NaN
242056,OL999983W,0,NaN,NaN


In [7]:
#save the summarized data to a new CSV file
work_ratings.to_csv("work_ratings_summary.csv", index=False)
